In [1]:
import csv
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import pandas as pd

In [ ]:
# The text fields are HUGE, max out the buffer
csv.field_size_limit(sys.maxsize)

file = open("hope.csv", "r")
csv_ = csv.reader(file, delimiter=",")
data = pd.DataFrame({
                      "title": [],
                      "text": []
                    })

#test = 50
indx = 0
for row in csv_:
    if indx != 0:
        new = pd.DataFrame({
                "title": [row[9]],
                "text": [row[7]]
              })

        data = data.append(new)
#    if indx >= test:
#        break
    indx += 1


vectorizer = TfidfVectorizer(stop_words="english", lowercase=True)
pca = PCA(n_components=8)

# Vectorize the text
tfidf = vectorizer.fit_transform(data.text).toarray()
tfidfDF = pd.DataFrame(tfidf)

# Join data
print("Nrows in data:", data.shape[0])
print("Nrows in tfidf:", tfidf.shape[0])
data.reset_index(drop=True, inplace=True)
tfidfDF.reset_index(drop=True, inplace=True)
newData = pd.concat([data, tfidfDF], axis=1)

# Perform PCA
preds = [name for name in list(newData) if type(name) is not str]
prComp = pd.DataFrame(pca.fit_transform(X = newData.ix[:, preds]))
newestData = pd.concat([newData.ix[:, "title"], prComp], axis=1)

# Write to file and close connection to old file
newestData.to_csv("wikiPCA.csv", sep=",", index=False)
file.close()